## Naibe Bayes (the easey way)

In [6]:
import os
import io
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

email_data = DataFrame({'message': [], 'class': []})

data = pd.concat([email_data, dataFrameFromDirectory('./emails/spam', 'spam')])
data = pd.concat([data, dataFrameFromDirectory('./emails/ham', 'ham')])


In [8]:
data.head()

,message,class
./emails/spam/00343.37d895b3a54847548875136ad6b0192d,Are you tired of spending a fortune on printer...,spam
./emails/spam/00045.7282c2c4e009744f2f3450d370009235,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
./emails/spam/00251.6b4b7e79e1706156839a00817d774e37,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",spam
./emails/spam/00447.bd5eb01e94f6d127465bf325513b2516,We guarantee you signups before you ever pay\n...,spam
./emails/spam/00398.1939605e3c713ff2ef852b1fbf10b0bb,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam


In [17]:
vectorizer =  CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values

classifier.fit(counts, targets)

MultinomialNB()

In [26]:
examples = ['Free Viagra now!!!', "Hi, Gabriel. How about a chess game tomorrow??", "free web site to watch movies online"]
example_counts = vectorizer.transform(examples)

predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham', 'spam'], dtype='<U4')